In [103]:
# Importuje funkcję do pobierania danych z Kaggle
from scripts.download_kaggle import download_data

# Pobiera dane z podanego zbioru na Kaggle
download_data("joannanplkrk/its-raining-cats")

Dataset URL: https://www.kaggle.com/datasets/joannanplkrk/its-raining-cats
Dataset downloaded to: data


In [104]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from joblib import dump


In [105]:
# Wczytanie danych z pliku CSV z użyciem średnika jako separatora
df = pd.read_csv('data/cat_breeds_dirty.csv', sep=';')

# Wyświetlenie pierwszych 5 wierszy danych
df.head()


,Breed,Age_in_years,Age_in_months,Gender,Neutered_or_spayed,Body_length,Weight,Fur_colour_dominant,Fur_pattern,Eye_colour,Allowed_outdoor,Preferred_food,Owner_play_time_minutes,Sleep_time_hours,Country,Latitude,Longitude
0,Angora,0.25,3.0,female,False,19.0,2.0,white,solid,blue,FALSE,wet,46.0,16.0,France,43.296482,5.369780
1,Angora,0.33,4.0,male,False,19.0,2.5,white,solid,blue,FALSE,wet,48.0,16.0,France,43.611660,3.877710
2,Angora,0.50,NaN,NaN,False,20.0,2.8,what does it mean dominant?,solid,green,I never allow my kitty outside!!!!!,wet,41.0,11.0,France,44.837789,-0.579180
3,Ankora,0.50,NaN,NaN,False,21.0,3.0,white,dirty,blue,FALSE,wet,24.0,8.0,France,43.611660,3.877710
4,Angora,0.50,NaN,NaN,NaN,21.0,3.0,red/cream,tabby,green,FALSE,wet,51.0,10.0,france,48.864716,2.349014


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Breed                    991 non-null    object 
 1   Age_in_years             1072 non-null   float64
 2   Age_in_months            1066 non-null   float64
 3   Gender                   1036 non-null   object 
 4   Neutered_or_spayed       1050 non-null   object 
 5   Body_length              1077 non-null   float64
 6   Weight                   1077 non-null   float64
 7   Fur_colour_dominant      1090 non-null   object 
 8   Fur_pattern              1055 non-null   object 
 9   Eye_colour               1064 non-null   object 
 10  Allowed_outdoor          1060 non-null   object 
 11  Preferred_food           1082 non-null   object 
 12  Owner_play_time_minutes  1082 non-null   float64
 13  Sleep_time_hours         1062 non-null   float64
 14  Country                 

In [107]:
# Usunięcie duplikatów z ramki danych
df.drop_duplicates(inplace=True)

In [108]:
# Usunięcie kolumn zawierających wyłącznie wartości null
df.dropna(axis=1, how='all', inplace=True)

In [109]:
# Usunięcie wierszy zawierających wyłącznie wartości null
df.dropna(how='all', inplace=True)

In [110]:
# Zamienia wartości tekstowe na ich logiczne odpowiedniki lub None
df.replace({
    'TRUE': True,
    'FALSE': False,
    'NA': None,
    '': None,
    ' ': None
}, inplace=True)


In [111]:
# Zlicza liczbę brakujących wartości w każdej kolumnie
df.isnull().sum()

Breed                      112
Age_in_years                31
Age_in_months               37
Gender                      67
Neutered_or_spayed          53
Body_length                 26
Weight                      26
Fur_colour_dominant         13
Fur_pattern                 48
Eye_colour                  39
Allowed_outdoor             43
Preferred_food              21
Owner_play_time_minutes     21
Sleep_time_hours            41
Country                     75
Latitude                    61
Longitude                   61
dtype: int64

In [112]:
df.shape

(1071, 17)

In [113]:
df.head()

,Breed,Age_in_years,Age_in_months,Gender,Neutered_or_spayed,Body_length,Weight,Fur_colour_dominant,Fur_pattern,Eye_colour,Allowed_outdoor,Preferred_food,Owner_play_time_minutes,Sleep_time_hours,Country,Latitude,Longitude
0,Angora,0.25,3.0,female,False,19.0,2.0,white,solid,blue,False,wet,46.0,16.0,France,43.296482,5.369780
1,Angora,0.33,4.0,male,False,19.0,2.5,white,solid,blue,False,wet,48.0,16.0,France,43.611660,3.877710
2,Angora,0.50,NaN,NaN,False,20.0,2.8,what does it mean dominant?,solid,green,I never allow my kitty outside!!!!!,wet,41.0,11.0,France,44.837789,-0.579180
3,Ankora,0.50,NaN,NaN,False,21.0,3.0,white,dirty,blue,False,wet,24.0,8.0,France,43.611660,3.877710
4,Angora,0.50,NaN,NaN,NaN,21.0,3.0,red/cream,tabby,green,False,wet,51.0,10.0,france,48.864716,2.349014


In [114]:
# Konwertuje kolumnę 'Age_in_years' na typ liczbowy; błędne wartości zamienia na NaN
df['Age_in_years'] = pd.to_numeric(df['Age_in_years'], errors='coerce')

# Konwertuje kolumnę 'Age_in_months' na typ liczbowy; błędne wartości zamienia na NaN
df['Age_in_months'] = pd.to_numeric(df['Age_in_months'], errors='coerce')

# Uzupełnia brakujące wartości w 'Age_in_months' na podstawie 'Age_in_years'
df.loc[df['Age_in_months'].isna(), 'Age_in_months'] = df['Age_in_years'] * 12


In [115]:
df.head()

,Breed,Age_in_years,Age_in_months,Gender,Neutered_or_spayed,Body_length,Weight,Fur_colour_dominant,Fur_pattern,Eye_colour,Allowed_outdoor,Preferred_food,Owner_play_time_minutes,Sleep_time_hours,Country,Latitude,Longitude
0,Angora,0.25,3.0,female,False,19.0,2.0,white,solid,blue,False,wet,46.0,16.0,France,43.296482,5.369780
1,Angora,0.33,4.0,male,False,19.0,2.5,white,solid,blue,False,wet,48.0,16.0,France,43.611660,3.877710
2,Angora,0.50,6.0,NaN,False,20.0,2.8,what does it mean dominant?,solid,green,I never allow my kitty outside!!!!!,wet,41.0,11.0,France,44.837789,-0.579180
3,Ankora,0.50,6.0,NaN,False,21.0,3.0,white,dirty,blue,False,wet,24.0,8.0,France,43.611660,3.877710
4,Angora,0.50,6.0,NaN,NaN,21.0,3.0,red/cream,tabby,green,False,wet,51.0,10.0,france,48.864716,2.349014


In [116]:
df.isnull().sum()

Breed                      112
Age_in_years                31
Age_in_months                4
Gender                      67
Neutered_or_spayed          53
Body_length                 26
Weight                      26
Fur_colour_dominant         13
Fur_pattern                 48
Eye_colour                  39
Allowed_outdoor             43
Preferred_food              21
Owner_play_time_minutes     21
Sleep_time_hours            41
Country                     75
Latitude                    61
Longitude                   61
dtype: int64

In [117]:
# Usuwa kolumnę 'Age_in_years' z ramki danych
df.drop(columns='Age_in_years', inplace=True)

In [118]:
df.isnull().sum()

Breed                      112
Age_in_months                4
Gender                      67
Neutered_or_spayed          53
Body_length                 26
Weight                      26
Fur_colour_dominant         13
Fur_pattern                 48
Eye_colour                  39
Allowed_outdoor             43
Preferred_food              21
Owner_play_time_minutes     21
Sleep_time_hours            41
Country                     75
Latitude                    61
Longitude                   61
dtype: int64

In [119]:
# Poprawia literówkę w nazwie rasy kota – zamienia 'Ankora' na 'Angora'
df['Breed'] = df['Breed'].replace({'Ankora': 'Angora'})

In [120]:
# Zastępuje niejednoznaczną wartość tekstową wartością NaN w kolumnie 'Fur_colour_dominant'
df['Fur_colour_dominant'] = df['Fur_colour_dominant'].replace({
    'what does it mean dominant?': np.nan
})

In [121]:
# Standaryzuje wartości w kolumnie 'Allowed_outdoor' do typu logicznego
# Zamienia tekst 'FALSE' i 'I NEVER ALLOW MY KITTY OUTSIDE!!!!!' na False,
# 'TRUE' na True, a pozostałe wartości na NaN
df['Allowed_outdoor'] = df['Allowed_outdoor'].apply(
    lambda x: False if str(x).strip().upper() in ['FALSE', 'I NEVER ALLOW MY KITTY OUTSIDE!!!!!']
    else True if str(x).strip().upper() == 'TRUE' else np.nan
).astype('bool', errors='ignore')

In [122]:
# Standaryzuje wartości w kolumnie 'Neutered_or_spayed' do typu logicznego
# Zamienia 'FALSE' na False, 'TRUE' na True, a pozostałe wartości na NaN
df['Neutered_or_spayed'] = df['Neutered_or_spayed'].apply(
    lambda x: False if str(x).strip().upper() == 'FALSE'
    else True if str(x).strip().upper() == 'TRUE' else np.nan
).astype('bool', errors='ignore')

In [123]:
# Zmienia format tekstu w kolumnie 'Country' – pierwsza litera wielka, reszta małe
df['Country'] = df['Country'].str.capitalize()

In [124]:
print("Braki danych po czyszczeniu:")
df.isna().sum()

Braki danych po czyszczeniu:


Breed                      112
Age_in_months                4
Gender                      67
Neutered_or_spayed           0
Body_length                 26
Weight                      26
Fur_colour_dominant         14
Fur_pattern                 48
Eye_colour                  39
Allowed_outdoor              0
Preferred_food              21
Owner_play_time_minutes     21
Sleep_time_hours            41
Country                     75
Latitude                    61
Longitude                   61
dtype: int64

In [125]:
# Lista kolumn tekstowych do oczyszczenia
tekstowe = ['Breed', 'Gender', 'Neutered_or_spayed', 'Fur_colour_dominant',
            'Fur_pattern', 'Eye_colour', 'Allowed_outdoor', 'Preferred_food', 'Country']

for col in tekstowe:
    if col in df.columns:
        # Konwertuje wartości na tekst i usuwa nadmiarowe spacje
        df[col] = df[col].astype(str).str.strip()
        # Zastępuje tekstowe 'nan' oraz puste ciągi wartością NaN
        df[col] = df[col].replace({'nan': np.nan, '': np.nan})
        # Ujednolica zapis – wszystkie litery małe
        df[col] = df[col].str.lower()

In [126]:
# Kolumna zawiera różne formy zapisu wartości logicznych, np. true/false, tak/nie, yes/no
if 'Neutered_or_spayed' in df.columns:
    # Zamienia tekstowe odpowiedniki na wartości logiczne
    df['Neutered_or_spayed'] = df['Neutered_or_spayed'].replace({
        'true': True, 'false': False,
        'tak': True, 'nie': False,
        'yes': True, 'no': False
    })
    # Konwertuje kolumnę do typu logicznego z obsługą NaN
    df['Neutered_or_spayed'] = df['Neutered_or_spayed'].astype('boolean')

C:\Users\Kamil\AppData\Local\Temp\ipykernel_17200\2052082699.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Neutered_or_spayed'] = df['Neutered_or_spayed'].replace({


In [127]:
# Poprawa typu logicznego w kolumnie 'Allowed_outdoor'
# Zamienia różne tekstowe formy wartości logicznych na odpowiedniki True/False
if 'Allowed_outdoor' in df.columns:
    df['Allowed_outdoor'] = df['Allowed_outdoor'].replace({
        'true': True, 'false': False,
        'tak': True, 'nie': False,
        'yes': True, 'no': False
    })
    # Konwertuje kolumnę do typu logicznego z obsługą wartości NaN
    df['Allowed_outdoor'] = df['Allowed_outdoor'].astype('boolean')

C:\Users\Kamil\AppData\Local\Temp\ipykernel_17200\622510739.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Allowed_outdoor'] = df['Allowed_outdoor'].replace({


In [128]:
# Ujednolicenie wartości w kolumnie 'Gender'
# Zamienia różne formy oznaczeń płci na standardowe 'male' i 'female'
if 'Gender' in df.columns:
    df['Gender'] = df['Gender'].replace({
        'male': 'male', 'female': 'female',
        'm': 'male', 'f': 'female',
        'samiec': 'male', 'samica': 'female'
    })
    # Uzupełnia brakujące wartości etykietą 'unknown'
    df['Gender'] = df['Gender'].fillna('unknown')

In [129]:
# Lista kolumn numerycznych wymagających uzupełnienia braków
numeryczne = ['Age_in_years', 'Age_in_months', 'Body_length', 'Weight',
              'Owner_play_time_minutes', 'Sleep_time_hours', 'Latitude', 'Longitude']

# Tworzy obiekt imputera, który uzupełnia braki medianą
imputer_num = SimpleImputer(strategy='median')

for col in numeryczne:
    if col in df.columns:
        # Uzupełnia brakujące wartości w kolumnie medianą
        df[col] = imputer_num.fit_transform(df[[col]])

In [130]:
#Obsługa brakujących danych w kolumnach kategorycznych
kategoryczne = ['Breed', 'Fur_colour_dominant', 'Fur_pattern', 'Eye_colour',
                'Preferred_food', 'Country']

for col in kategoryczne:
    if col in df.columns:
        # Uzupełnia brakujące wartości etykietą 'unknown'
        df[col] = df[col].fillna('unknown')

In [131]:
# Tworzy obiekt skalera MinMax do normalizacji wartości w zakresie [0, 1]
scaler = MinMaxScaler()

# Lista kolumn do normalizacji
do_normalizacji = ['Body_length', 'Weight', 'Owner_play_time_minutes', 'Sleep_time_hours', 'Age_total_months']

# Filtrowanie – tylko istniejące w ramce danych kolumny
do_norm2 = [col for col in do_normalizacji if col in df.columns]

# Dopasowuje skalera i normalizuje wybrane kolumny
df[do_norm2] = scaler.fit_transform(df[do_norm2])

In [132]:
# Zamienia wartości tekstowe w wybranych kolumnach na liczby za pomocą label encodingu
label_enc_cols = ['Breed', 'Fur_colour_dominant', 'Fur_pattern', 'Eye_colour',
                  'Preferred_food', 'Country', 'Gender']

for col in label_enc_cols:
    if col in df.columns:
        le = LabelEncoder()
        # Dopasowuje encoder i przekształca wartości tekstowe na liczby całkowite
        df[col] = le.fit_transform(df[col])

In [133]:
print("Finalne info o DataFrame:")
print(df.info())
print("\nKilka pierwszych wierszy po oczyszczeniu i normalizacji:")
print(df.head())

Finalne info o DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1071 entries, 0 to 1070
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Breed                    1071 non-null   int64  
 1   Age_in_months            1071 non-null   float64
 2   Gender                   1071 non-null   int64  
 3   Neutered_or_spayed       1071 non-null   boolean
 4   Body_length              1071 non-null   float64
 5   Weight                   1071 non-null   float64
 6   Fur_colour_dominant      1071 non-null   int64  
 7   Fur_pattern              1071 non-null   int64  
 8   Eye_colour               1071 non-null   int64  
 9   Allowed_outdoor          1071 non-null   boolean
 10  Preferred_food           1071 non-null   int64  
 11  Owner_play_time_minutes  1071 non-null   float64
 12  Sleep_time_hours         1071 non-null   float64
 13  Country                  1071 non-null   int64  
 14  Lat

In [134]:
# Zapisuje oczyszczony i znormalizowany zbiór danych do pliku CSV bez indeksu
df.to_csv('cat_breeds_clean_normalized.csv', index=False)

In [135]:
df.isnull().sum()


Breed                      0
Age_in_months              0
Gender                     0
Neutered_or_spayed         0
Body_length                0
Weight                     0
Fur_colour_dominant        0
Fur_pattern                0
Eye_colour                 0
Allowed_outdoor            0
Preferred_food             0
Owner_play_time_minutes    0
Sleep_time_hours           0
Country                    0
Latitude                   0
Longitude                  0
dtype: int64

In [136]:
# Sprawdza, czy kolumna 'Breed' (target) istnieje w danych
if 'Breed' not in df.columns:
    raise ValueError("Nie znaleziono kolumny 'Breed' w DataFrame!")

# Oddziela dane wejściowe (X) od etykiet (y)
X = df.drop(columns=['Breed'])
y = df['Breed']

# Jeśli w danych znajdują się kolumny bez znaczenia dla modelu, usuń je w tym miejscu
# X = X.drop(columns=['SomeID', 'Owner_name', ...], errors='ignore')

# Dzieli dane na zbiór treningowy i testowy z zachowaniem proporcji klas
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,      # 25% danych zostaje przeznaczone na test
    random_state=42,     # zapewnia powtarzalność podziału
    stratify=y           # utrzymuje rozkład klas jak w oryginalnych danych
)

In [137]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

In [138]:
# Słowniki do przechowywania wytrenowanych modeli i ich wyników
trained_models = {}
results = {}

# Iteracja po zdefiniowanych modelach
for name, model in models.items():
    print(f"\n--- Trenowanie modelu: {name} ---")
    
    # Trenowanie modelu na zbiorze treningowym
    model.fit(X_train, y_train)
    
    # Predykcja na zbiorze testowym
    y_pred = model.predict(X_test)
    
    # Obliczenie dokładności predykcji
    acc = accuracy_score(y_test, y_pred)
    print(f"Dokładność (accuracy) na zbiorze testowym: {acc:.4f}")
    
    # Wyświetlenie macierzy pomyłek
    print("Macierz pomyłek (confusion matrix):")
    print(confusion_matrix(y_test, y_pred))
    
    # Wyświetlenie szczegółowego raportu klasyfikacji
    print("Raport klasyfikacji (classification report):")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # Zapis wytrenowanego modelu i jego dokładności
    trained_models[name] = model
    results[name] = acc


--- Trenowanie modelu: LogisticRegression ---


C:\Users\Kamil\PycharmProjects\ewd-project-2025\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Dokładność (accuracy) na zbiorze testowym: 0.7388
Macierz pomyłek (confusion matrix):
[[49  1  3  0  0  1  8  0  0]
 [ 3  0  0  0  0  0  0  0  0]
 [ 5  0 60  0  2  0  1  0  0]
 [ 0  0  2  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0]
 [ 5  0  1  0  0  0 89  0  0]
 [ 8  0 12  0  0  0  8  0  0]
 [ 0  0  0  0  0  0  2  0  0]]
Raport klasyfikacji (classification report):
              precision    recall  f1-score   support

           0       0.70      0.79      0.74        62
           1       0.00      0.00      0.00         3
           2       0.71      0.88      0.78        68
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         1
           6       0.82      0.94      0.87        95
           7       0.00      0.00      0.00        28
           8       0.00      0.00      0.00         2

    accuracy                           0.74       268
  

In [139]:
# Wyświetla podsumowanie dokładności wszystkich wytrenowanych modeli
print("\n=== Podsumowanie dokładności modeli ===")
for name, acc in results.items():
    print(f"{name:20s} : {acc:.4f}")


=== Podsumowanie dokładności modeli ===
LogisticRegression   : 0.7388
RandomForest         : 0.8209
GradientBoosting     : 0.8097
KNN                  : 0.6754
SVM                  : 0.3955


In [140]:
# Wybiera model o najwyższej dokładności
best_name = max(results, key=results.get)
best_model = trained_models[best_name]
print(f"Najlepszy model: {best_name} (accuracy = {results[best_name]:.4f})")

# Zapisuje najlepszy wytrenowany model do pliku
dump(best_model, 'best_cat_classifier.pkl')
print("Zapisano najlepszy model do pliku: best_cat_classifier.pkl")


Najlepszy model: RandomForest (accuracy = 0.8209)
Zapisano najlepszy model do pliku: best_cat_classifier.pkl


In [141]:
# Pobiera wszystkie parametry najlepszego modelu
best_params = best_model.get_params()

# Tworzy nową instancję tego samego typu modelu z tymi samymi parametrami
ModelClass = type(best_model)
best_full = ModelClass(**best_params)

In [142]:
# Trenuje nową instancję najlepszego modelu na pełnym zbiorze danych
best_full.fit(X, y)

# Zapisuje w pełni wytrenowany model do pliku
dump(best_full, 'best_cat_classifier_full.pkl')
print("Dotrenowano najlepszy model na pełnym zbiorze i zapisano jako: best_cat_classifier_full.pkl")


Dotrenowano najlepszy model na pełnym zbiorze i zapisano jako: best_cat_classifier_full.pkl
